In [1]:
import fnmatch, os, sys
# sys.path.append('/Users/sherryan/Desktop/cage_data/')
# print(sys.path)
import cage_data
import numpy as np
import pickle as pickle

In [2]:
# data_path = '/Users/sherryan/ssa-pop/'
data_path = "/Users/andrewshen/Desktop/neural_decoding/data/"
# files_name = ['Pop_20210709_Cage_001.pkl']
files_name = ["Pop_20210709_Cage_004.pkl"]
cage_data_list = []
for file in files_name:
    print('The file %s is going to be loaded'%(data_path + file))
    with open ( data_path + file, 'rb' ) as fp:
        cage_data = pickle.load(fp)
        cage_data_list.append(cage_data)


The file /Users/andrewshen/Desktop/neural_decoding/data/Pop_20210709_Cage_004.pkl is going to be loaded


In [ ]:
# Andrew's code
my_cage_data = cage_data_list[0]

In [ ]:
bin_size = .025

In [ ]:
keys = cage_data.behave_event.keys()

In [ ]:
def find_start_end_event(timing, cage_data):
    timeframe = cage_data.binned['timeframe']
    segment_range = np.where((timeframe>=timing-0.5) & (timeframe<=timing+1))[0]
    start_idx = segment_range[0]
    end_idx = segment_range[-1]
    return start_idx, end_idx

In [ ]:
behav_dict = {'crawl': [], 'precision': [], 'power': []}

In [ ]:
# Andrew's code

# To get the binned spike counts
binned_spike_counts = my_cage_data.binned['spikes']

# To get the rectified, filtered and downsampled EMGs
filtered_EMG = my_cage_data.binned['filtered_EMG']

# To get the time frame of the binned data
timeframe = my_cage_data.binned['timeframe']

m1_data = np.transpose(binned_spike_counts)
emg_data = np.transpose(filtered_EMG)

In [ ]:
concat_spikes = []
curr_idx = 0
for key in keys:  
    if key == 'bar_touch':
        event = 'crawl'
    if key == 'treat_touch':
        event = 'precision'
    if key == 'pg_force_onset':
        event = 'power'
    behav_dict[event].append(curr_idx)
    for cage_data in cage_data_list:
        spikes = np.array(cage_data.binned['spikes'])
        for timing in cage_data.behave_event[key]:
            start_idx, end_idx = find_start_end_event(timing, cage_data)
            curr_idx += (end_idx - start_idx + 1)
            concat_spikes.append(spikes[:,start_idx:end_idx+1])
            behav_dict[event].append(curr_idx)



In [25]:
# Andrew's code to keep only timestamps Set2 labeled behaviors for M1 and EMG

m1_data_labeled_set2 = []
emg_data_labeled_set2 = []
behavioral_labels_set2 = []

concat_spikes = []
curr_idx = 0
for key in keys:  
    if key == 'bar_touch':
        event = 'crawl'
    if key == 'treat_touch':
        event = 'precision'
    if key == 'pg_force_onset':
        event = 'power'
    behav_dict[event].append(curr_idx)
    for cage_data in cage_data_list:
        spikes = np.array(cage_data.binned['spikes'])
        for timing in cage_data.behave_event[key]:
            start_idx, end_idx = find_start_end_event(timing, cage_data)
            curr_idx += (end_idx - start_idx + 1)
            concat_spikes.append(spikes[:,start_idx:end_idx+1])

            # Andrew's code # TODO: CURRENT: format data for set2 labels
            behavioral_label = event
            if len(m1_data_labeled_set2) == 0:
                m1_data_labeled_set2 = m1_data[start_idx:end_idx+1]
                emg_data_labeled_set2 = emg_data[start_idx:end_idx+1]
                behavioral_labels_set2 = np.array([behavioral_label for i in range(end_idx-start_idx+1)])
            else:
                m1_data_labeled_set2 = np.concatenate((m1_data_labeled_set2, m1_data[start_idx:end_idx+1]))
                emg_data_labeled_set2 = np.concatenate((emg_data_labeled_set2, emg_data[start_idx:end_idx+1]))
                behavioral_labels_set2 = np.concatenate((behavioral_labels_set2, np.array([behavioral_label for i in range(end_idx-start_idx+1)])))

# Save M1, EMG, and Set2 behavioral labels
out_path = "/Users/andrewshen/Desktop/neural_decoding/data/"
np.save(out_path + "set2_data/m1_set2", m1_data_labeled_set2)
np.save(out_path + "set2_data/emg_set2", emg_data_labeled_set2)
np.save(out_path + "set2_data/behavioral_set2", behavioral_labels_set2)

In [13]:
behav_dict

{'crawl': [0,
  60,
  120,
  180,
  240,
  300,
  360,
  420,
  480,
  540,
  600,
  660,
  720,
  780,
  840,
  900,
  960,
  1020,
  1080,
  1140,
  1200,
  1260,
  1320,
  1380,
  1440,
  1500,
  1560,
  1620,
  1680,
  1740,
  1800,
  1860,
  1920,
  1980,
  2040,
  2100,
  2160,
  2220,
  2280,
  2340,
  2400,
  2460,
  2520,
  2580,
  2640,
  2700,
  2760,
  2820,
  2880,
  2940,
  3000,
  3060,
  3120,
  3180,
  3240,
  3300,
  3360],
 'precision': [3360,
  3420,
  3480,
  3540,
  3600,
  3660,
  3720,
  3780,
  3840,
  3900,
  3960,
  4020,
  4080,
  4140,
  4200,
  4260,
  4320,
  4380,
  4440,
  4500,
  4560,
  4620,
  4680,
  4740,
  4800,
  4860,
  4920,
  4980,
  5040,
  5100,
  5160,
  5220],
 'power': [5220,
  5280,
  5340,
  5400,
  5460,
  5520,
  5580,
  5640,
  5700,
  5760,
  5820,
  5880]}